In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-156528", resource_group = 'aml-quickstarts-156528', subscription_id = 'd4ad7261-832d-46b2-b093-22156001df5b', location = 'South Central US')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-156528
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-156528


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import os
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME','test-cluster')
compute_min_nodes = os.environ.get('AML_COMPUTE_MIN_NODES',0)
compute_max_nodes = os.environ.get('AML_COMPUTE_MAX_NODES',4)
vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU','STANDARD_D2_V2')

# Check if cluster exists already
try:
    compute_target = ComputeTarget(workspace = ws, name = cluster_name)
    print('Found existing compute cluster, just  it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                           min_nodes = compute_min_nodes,
                                                           max_nodes = compute_max_nodes)
    
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
    compute_target.wait_for_completion(show_output = True)

Found existing compute cluster, just  it


In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal, HyperDriveRun
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice

from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--kernel": choice('linear', 'rbf', 'poly', 'sigmoid'),
        "--penalty": choice(0.5, 1, 1.5)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# Deprecated
#est = SKLearn(source_directory = './',
              #compute_target = compute_target,
              #entry_script = 'train.py',
              #)

src = ScriptRunConfig(source_directory = './',
                       script = 'train.py',
                       compute_target = compute_target
                       )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling = ps,
                                     policy = policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 20,
                                     max_concurrent_runs = 4)

In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_exp = exp.submit(config = hyperdrive_config)

RunDetails(hyperdrive_exp).show()

hyperdrive_exp.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d49c5341-ffca-4529-a734-efeaac6d70c8
Web View: https://ml.azure.com/runs/HD_d49c5341-ffca-4529-a734-efeaac6d70c8?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-156528/workspaces/quick-starts-ws-156528&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-09-01T06:01:24.056444][API][INFO]Experiment created<END>\n""<START>[2021-09-01T06:01:24.601331][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-09-01T06:01:24.770844][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d49c5341-ffca-4529-a734-efeaac6d70c8
Web View: https://ml.azure.com/runs/HD_d49c5341-ffca-4529-a734-efeaac6d70c8?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-156528/workspaces/quick-starts-ws-156528&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_d49c5341-ffca-4529-a734-efeaac6d70c8',
 'target': 'test-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-09-01T06:01:23.794764Z',
 'endTimeUtc': '2021-09-01T06:14:24.834958Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '24cdc492-f53e-431a-9343-645711716ada',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg156528.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_d49c5341-ffca-4529-a

KeyError: 'log_files'

In [ ]:
# Get your best run and save the model from that run.
# Once all of the hyperparameter tuning runs have completed, identify the best performing configuration and hyperparameter values:

## Find and return the Run instance that corresponds to the best performing run amongst all child runs.
best_run = hyperdrive_exp.get_best_run_by_primary_metric()

## Return the metrics from all the runs that were launched by this HyperDriveRun.
best_run_metrics = best_run.get_metrics()

## Returns a dictionary with details for the run
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])


### Saving the best model ###
import joblib
joblib.dump(best_run, 'hyperd_bank_mkt_sklearn.joblib')
#model = best_run.register_model(model_name='hyperd_bank_mkt_sklearn', model_path='outputs/hyperd_bank_mkt_sklearn.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric='Accuracy',
    training_data=ds,
    label_column_name=y,
    n_cross_validations=2)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
## Below we select the best pipeline from our iterations. 
# The get_output method returns the best run and the fitted model. 
# Overloads on get_output allow you to retrieve the best run and fitted model 
# for any logged metric or for a particular iteration.
best_run, fitted_model = automl_run.get_output()
model_name = best_run.properties['model_name']

description = 'AutoML Model trained on bank marketing data to predict if a client will subscribe to a term deposit'
tags = None
model = automl_run.register_model(model_name = model_name, description = description, tags = tags)


In [ ]:
cpu_cluster.delete()